In [1]:
# Cell 1: Clone STG-NF và Alpha Pose
!git clone https://github.com/orhir/STG-NF.git /kaggle/working/STG-NF
!git clone https://github.com/MVIG-SJTU/AlphaPose.git /kaggle/working/AlphaPose

Cloning into '/kaggle/working/STG-NF'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (449/449), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 449 (delta 284), reused 423 (delta 265), pack-reused 0 (from 0)
Receiving objects: 100% (449/449), 827.77 KiB | 14.03 MiB/s, done.
Resolving deltas: 100% (284/284), done.
Cloning into '/kaggle/working/AlphaPose'...
remote: Enumerating objects: 2749, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 2749 (delta 4), reused 0 (delta 0), pack-reused 2740 (from 2)
Receiving objects: 100% (2749/2749), 118.82 MiB | 24.98 MiB/s, done.
Resolving deltas: 100% (1379/1379), done.


In [2]:
%cd /kaggle/working/AlphaPose
!git clone https://github.com/YuliangXiu/PoseFlow.git

/kaggle/working/AlphaPose
Cloning into 'PoseFlow'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 165 (delta 33), reused 23 (delta 7), pack-reused 110 (from 1)
Receiving objects: 100% (165/165), 9.52 MiB | 27.23 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [3]:
%cd /kaggle/working/AlphaPose

# Fix numpy trước
!pip install "numpy==1.26.4"

# Cài các lib cần thiết
!pip install -U "Cython==0.29.36"

# Build AlphaPose
!python setup.py build develop

/kaggle/working/AlphaPose
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 72.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires jupyter-server==2.14.0, but you have jupyter-server 2.12.5 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is inco

In [4]:
# Cell 4: Download YOLO detector (quan trọng!)
import os
from pathlib import Path

yolo_dir = Path("/kaggle/working/AlphaPose/detector/yolo/data")
yolo_dir.mkdir(parents=True, exist_ok=True)

print("Downloading YOLOv3 weights...")
os.chdir(yolo_dir)
!wget -q https://pjreddie.com/media/files/yolov3-spp.weights

print(f"YOLO weights saved to: {yolo_dir}")
print(f"Files: {list(yolo_dir.glob('*'))}")

os.chdir("/kaggle/working/AlphaPose")

YOLO weights saved to: /kaggle/working/AlphaPose/detector/yolo/data
Files: [PosixPath('/kaggle/working/AlphaPose/detector/yolo/data/yolov3-spp.weights')]


In [5]:
# Cell 5: Verify YOLO detector
from pathlib import Path

yolo_weight = Path("/kaggle/working/AlphaPose/detector/yolo/data/yolov3-spp.weights")
if yolo_weight.exists():
    print(f"✅ YOLO weight found: {yolo_weight}")
    print(f"   Size: {yolo_weight.stat().st_size / (1024**2):.1f} MB")
else:
    print("❌ YOLO weight NOT found! AlphaPose sẽ không detect được người.")
    print(f"   Expected path: {yolo_weight}")

# Check config files
cfg_files = [
    "/kaggle/working/AlphaPose/configs/coco/resnet/256x192_res152_lr1e-3_1x-duc.yaml",
    "/kaggle/working/AlphaPose/detector/yolo/cfg/yolov3-spp.cfg"
]

for cfg in cfg_files:
    if Path(cfg).exists():
        print(f"✅ Config found: {cfg}")
    else:
        print(f"❌ Config NOT found: {cfg}")

✅ YOLO weight found: /kaggle/working/AlphaPose/detector/yolo/data/yolov3-spp.weights
   Size: 240.5 MB
✅ Config found: /kaggle/working/AlphaPose/configs/coco/resnet/256x192_res152_lr1e-3_1x-duc.yaml
✅ Config found: /kaggle/working/AlphaPose/detector/yolo/cfg/yolov3-spp.cfg


In [11]:
# Cell 6: Chạy AlphaPose với YOLO detector
import os
from pathlib import Path
import shutil
import subprocess
from subprocess import TimeoutExpired

# ================== CẤU HÌNH CHUNG ==================
alphapose_root = Path("/kaggle/working/AlphaPose")

# SOURCE: folder chứa video
videos_dir = Path("/kaggle/input/ubnormal/Scene1")

# Output chung
out_root = Path("/kaggle/working/alphapose_out")
out_root.mkdir(parents=True, exist_ok=True)

# Config paths
cfg_path = alphapose_root / "configs/coco/resnet/256x192_res152_lr1e-3_1x-duc.yaml"
checkpoint_path = Path("/kaggle/input/pretrained-models/fast_421_res152_256x192.pth")

# YOLO detector config (QUAN TRỌNG!)
detector_cfg = alphapose_root / "detector/yolo/cfg/yolov3-spp.cfg"
detector_weights = alphapose_root / "detector/yolo/data/yolov3-spp.weights"

VIDEO_EXTS = {".mp4", ".avi", ".mov", ".mkv", ".mpeg", ".mpg", ".m4v"}

# ================== HÀM CHẠY ALPHAPOSE ==================
def run_alphapose_video(video_path: Path, out_json_path: Path, timeout_sec: int = 600):
    """
    Chạy AlphaPose cho 1 video với YOLO detector.
    """
    vid_name = video_path.stem
    temp_out = Path(f"/kaggle/working/alphapose_temp_video_{vid_name}")
    temp_out.mkdir(parents=True, exist_ok=True)

    print(f"\n=========== Đang chạy VIDEO: {video_path.name} ===========")

    # Command với đầy đủ tham số detector
    cmd = [
        "python",
        "scripts/demo_inference.py",
        "--cfg", str(cfg_path),
        "--checkpoint", str(checkpoint_path),
        "--outdir", str(temp_out),
        "--video", str(video_path),
        "--detector", "yolo",  # Chỉ định detector
        "--detfile", "",  # Để trống = tự động detect
        "--save_video",  # Lưu video để debug
    ]
env = os.environ.copy()
env["PYTHONPATH"] = str(alphapose_root) + ":" + env.get("PYTHONPATH", "")

    try:
        # BẮT LOG để biết có lỗi gì
        proc = subprocess.run(
            cmd,
            cwd=str(alphapose_root),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            timeout=timeout_sec,
            check=False,
            env=env,
        )
    except TimeoutExpired:
        print(f"⚠ TIMEOUT ({timeout_sec}s), bỏ qua video {vid_name}")
        shutil.rmtree(temp_out, ignore_errors=True)
        return

    # In log nếu có lỗi
    if proc.returncode != 0:
        print(f"⚠ AlphaPose exit code = {proc.returncode}")
        print("----- STDERR (last 2000 chars) -----")
        print(proc.stderr[-2000:])
        print("----- STDOUT (last 2000 chars) -----")
        print(proc.stdout[-2000:])
    else:
        # In một vài dòng cuối để biết trạng thái
        stdout_lines = proc.stdout.split('\n')
        print("Last 5 lines of output:")
        for line in stdout_lines[-5:]:
            if line.strip():
                print("  ", line)

    # TÌM JSON THỰC TẾ
    json_candidates = sorted(temp_out.rglob("*.json"))

    # Ưu tiên file có chữ "alphapose" và "result"
    preferred = None
    for p in json_candidates:
        name = p.name.lower()
        if "alphapose" in name and "result" in name:
            preferred = p
            break

    chosen = preferred if preferred else (json_candidates[0] if json_candidates else None)

    if chosen and chosen.exists():
        # Kiểm tra xem file JSON có dữ liệu không
        import json
        try:
            with open(chosen, 'r') as f:
                data = json.load(f)
            if len(data) == 0:
                print(f"⚠ JSON rỗng (không detect được người): {chosen.name}")
            else:
                print(f"✅ Tìm thấy {len(data)} keypoints")
        except:
            print(f"⚠ Không đọc được JSON: {chosen.name}")
        
        out_json_path.parent.mkdir(parents=True, exist_ok=True)
        shutil.move(str(chosen), str(out_json_path))
        print(f"✅ Đã lưu: {out_json_path}")
    else:
        print(f"⚠ Không tìm thấy file .json nào trong: {temp_out}")
        # In cây thư mục để thấy AlphaPose tạo gì
        files = sorted([str(p.relative_to(temp_out)) for p in temp_out.rglob("*")])
        print("Nội dung temp_out:")
        for f in files[:50]:
            print(" -", f)
        if len(files) > 50:
            print(f"... ({len(files)-50} items nữa)")

    # Giữ lại video output để debug (comment dòng này nếu không cần)
    # shutil.rmtree(temp_out, ignore_errors=True)

# ================== CHẠY HẾT VIDEO TRONG FOLDER ==================
videos = sorted([p for p in videos_dir.iterdir() if p.is_file() and p.suffix.lower() in VIDEO_EXTS])

print("\n===== VIDEOS =====")
print("Source folder:", videos_dir)
print("Tổng số video:", len(videos))
print("Output folder:", out_root)
print("YOLO detector:", detector_weights.exists())

# Test với 1 video trước
if videos:
    test_video = videos[0]
    out_json = out_root / f"{test_video.stem}_alphapose_results.json"
    
    print(f"\n🧪 Testing với video đầu tiên: {test_video.name}")
    run_alphapose_video(test_video, out_json, timeout_sec=600)
    
    # Nếu thành công, chạy tiếp các video khác
    if out_json.exists():
        print("\n✅ Test thành công! Chạy tiếp các video khác...\n")
        for video_path in videos[1:]:
            vid_name = video_path.stem
            out_json = out_root / f"{vid_name}_alphapose_results.json"

            if out_json.exists():
                print(f"⏭ Skip (đã có): {out_json.name}")
                continue

            run_alphapose_video(video_path, out_json, timeout_sec=600)
    else:
        print("\n❌ Test không thành công. Kiểm tra lỗi ở trên.")

print("\n=== Hoàn tất AlphaPose ===")
print("JSON nằm trong:", out_root)

IndentationError: unexpected indent (3088289121.py, line 54)

In [7]:
# Cell 7: Kiểm tra kết quả
import json
from pathlib import Path

out_root = Path("/kaggle/working/alphapose_out")
json_files = sorted(out_root.glob("*.json"))

print(f"\n===== KẾT QUẢ ALPHAPOSE =====")
print(f"Tổng số file JSON: {len(json_files)}\n")

for jf in json_files:
    try:
        with open(jf, 'r') as f:
            data = json.load(f)
        print(f"✅ {jf.name}: {len(data)} keypoints")
    except Exception as e:
        print(f"❌ {jf.name}: Lỗi đọc file - {e}")


===== KẾT QUẢ ALPHAPOSE =====
Tổng số file JSON: 0



# POSEFLOW TRACKING

In [8]:
# Cell 8: PoseFlow tracking (chỉ chạy sau khi có JSON từ AlphaPose)
import os
import json
from pathlib import Path
import subprocess

alphapose_root = Path("/kaggle/working/AlphaPose")
poseflow_root = alphapose_root / "PoseFlow"

# Input: JSON từ AlphaPose
alphapose_json_dir = Path("/kaggle/working/alphapose_out")

# Output: JSON đã tracking
tracked_json_dir = Path("/kaggle/working/poseflow_final_json")
tracked_json_dir.mkdir(parents=True, exist_ok=True)

# Video source để PoseFlow visualize
videos_dir = Path("/kaggle/input/ubnormal/Scene1")

# Output visualize
vis_dir = Path("/kaggle/working/poseflow_vis")
vis_dir.mkdir(parents=True, exist_ok=True)

json_files = sorted(alphapose_json_dir.glob("*_alphapose_results.json"))

print(f"\n===== POSEFLOW TRACKING =====")
print(f"Tìm thấy {len(json_files)} file JSON từ AlphaPose\n")

for json_file in json_files:
    # Tìm video tương ứng
    vid_name = json_file.stem.replace("_alphapose_results", "")
    video_path = videos_dir / f"{vid_name}.mp4"
    
    if not video_path.exists():
        # Thử các extension khác
        for ext in [".avi", ".mov", ".mkv"]:
            alt_path = videos_dir / f"{vid_name}{ext}"
            if alt_path.exists():
                video_path = alt_path
                break
    
    if not video_path.exists():
        print(f"⚠ Không tìm thấy video cho {vid_name}, skip")
        continue
    
    # Output paths
    tracked_json = tracked_json_dir / f"{vid_name}_tracked.json"
    vis_output = vis_dir / f"{vid_name}_tracked.mp4"
    
    if tracked_json.exists():
        print(f"⏭ Skip (đã có): {tracked_json.name}")
        continue
    
    print(f"\n▶ Tracking: {vid_name}")
    
    # Chạy PoseFlow
    cmd = [
        "python",
        "PoseFlow/tracker-general.py",
        "--imgdir", str(video_path),
        "--in_json", str(json_file),
        "--out_json", str(tracked_json),
        "--visdir", str(vis_output.parent),
    ]
    
    try:
        result = subprocess.run(
            cmd,
            cwd=str(alphapose_root),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            timeout=300,
        )
        
        if result.returncode == 0:
            print(f"  ✅ Tracked: {tracked_json.name}")
        else:
            print(f"  ⚠ PoseFlow exit code: {result.returncode}")
            if result.stderr:
                print(f"  Error: {result.stderr[-500:]}")
    except subprocess.TimeoutExpired:
        print(f"  ⚠ Timeout (300s)")
    except Exception as e:
        print(f"  ⚠ Error: {e}")

print("\n=== Hoàn tất PoseFlow tracking ===")


===== POSEFLOW TRACKING =====
Tìm thấy 0 file JSON từ AlphaPose


=== Hoàn tất PoseFlow tracking ===


In [9]:
# Cell 9: ZIP kết quả
import zipfile
from pathlib import Path

FINAL_JSON_ROOT = Path("/kaggle/working/poseflow_final_json")
VIS_ROOT = Path("/kaggle/working/poseflow_vis")

OUT_ZIP_JSON = Path("/kaggle/working/poseflow_tracked_json.zip")
OUT_ZIP_VIS = Path("/kaggle/working/poseflow_visualize.zip")

def zip_folder(src_dir: Path, zip_path: Path):
    if not src_dir.exists():
        print(f"⚠ Folder không tồn tại: {src_dir}")
        return
    
    files = list(src_dir.rglob("*"))
    if not files:
        print(f"⚠ Folder trống: {src_dir}")
        return
    
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        for f in files:
            if f.is_file():
                zf.write(f, arcname=f.relative_to(src_dir))
    print(f"✅ Zipped: {zip_path} ({len(files)} files)")

zip_folder(FINAL_JSON_ROOT, OUT_ZIP_JSON)
zip_folder(VIS_ROOT, OUT_ZIP_VIS)

print("\n=== DONE ZIP ===")

⚠ Folder trống: /kaggle/working/poseflow_final_json
⚠ Folder trống: /kaggle/working/poseflow_vis

=== DONE ZIP ===
